In [10]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import math

import copy 

# Transformer model

In [3]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder 
        self.decoder = decoder
        self.src_embed = src_embed 
        self.tgt_embed = tgt_embed 
        self.generator = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(
            memory = self.encode(src, src_mask), 
            src_mask = src_mask,
            tgt=tgt, 
            tgt_mask=tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [4]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [7]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self, x):
        
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [44]:
def subsequent_mask(size):
    atten_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(atten_shape), diagonal=1)
    return subsequent_mask == 0 

## Encoder

In [6]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [8]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        for layer   in self.layers:
            x = layer(x, mask)
        
        return self.norm(x)

In [22]:
class SublayerConnection(nn.Module):
    """ A residual connection followed by a layer norm. """
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [23]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn 
        self.feed_forward = feed_forward 
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
    
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        
        return self.sublayer[1](x, self.feed_forward)

## Decoder

In [24]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        
        return self.norm(x)

In [25]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size 
        self.self_attn = self_attn 
        self.src_attn = src_attn 
        self.feed_forward = feed_forward 
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

## Attention

$$
\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

In [90]:
def attention(query, key, value, mask=None, dropout=None):
    # (2, 8, 10, 64)
    d_k = key.size(-1)
    # scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    scores = torch.einsum('bhqx, bhkx -> bhqk', query, key) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)

    # return torch.matmul(p_attn, value), p_attn
    return torch.einsum('bhqk, bhkv -> bhqv', p_attn, value), p_attn

In [91]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0, "d_model must be divisible by h"
        self.d_k = d_model // h 
        self.h = h 
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None 
        self.dropout = nn.Dropout(p=dropout)
        
    
    def forward(self, q, k, v, mask=None):
        print('p', q.size())
        nbatches = q.size(0)
        
        if mask is not None:
            mask = mask.unsqueeze(1)
            
        q, k, v = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (q, k, v))]
        
        # x: (B, H, S, D//H)
        x, self.attn = attention(q, k, v, mask=mask, dropout=self.dropout)
        
        
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        
        return self.linears[-1](x)
        

## Pointwise Feedforward Network

In [92]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.w_1(x)
        x = self.dropout(F.relu(x))
        x = self.w_2(x)
        return x

In [93]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [94]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + torch.autograd.Variable(self.pe[:, : x.size(1)], requires_grad=False)
        return self.dropout(x)

# Test Model

In [95]:
d_model = 512 
N = 6 
d_ff = 2048 
h =8 
dropout = 0.1
src_vocab = 10
tgt_vocab = 10


c = copy.deepcopy
attn = MultiHeadedAttention(h, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
position = PositionalEncoding(d_model, dropout)
model = EncoderDecoder(
    Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
    Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
    nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
    nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
    Generator(d_model, tgt_vocab)
    )

In [96]:
test_src = torch.randint(0, 10, (2, 10))
test_tgt = torch.randint(0, 10, (2, 10))

In [97]:
class Batch:
    "Object for holding a batch of data with mask during training."

    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & torch.autograd.Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
        )
        return tgt_mask

In [98]:
batch = Batch(test_src, test_tgt)

In [99]:
out = model(batch.src, batch.trg, batch.src_mask, batch.trg_mask)

p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 10, 512])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 9, 512])
torch.Size([2, 8, 9, 64])
torch.Size([2, 8, 9, 64])
torch.Size([2, 8, 9, 64])
p torch.Size([2, 9, 512])
torch.Size([2, 8, 9, 64])
torch.Size([2, 8, 10, 64])
torch.Size([2, 8, 10, 64])
p torch.Size([2, 9, 512])
torch.Size([2, 8, 9, 64])
torch.Size([2, 8, 9, 64])
torch.Size([2, 8, 9, 64])
p torch.Size([2, 9, 512])
torch.Size([

# Training 

In [ ]:
from ..utils.engine import train 
from ..utils.plots import visualize_attention

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super().__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size 
        self.true_dist = None
    
    def forward(self, x, target):
        assert x.size(1) == self.size
        
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        
        self.true_dist = true_dist
        return self.criterion(x, torch.autograd.Variable(true_dist, requires_grad=False))